<a href="https://colab.research.google.com/github/InternetGuru/gg/blob/main/Graph_Generalizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Little manual to use the program:
1. Use Set up in the 1st code cell to put the number of vertices and the list of facets 
2. Run the 1st code cell 
3. Find the desired property cell further down
4. Run that cell and check the result at the bottom of the cell

# Setup

In [19]:
from optparse import OptionParser
import os
import sys

# Deafults
list_of_facets = [[1,2], [1,3], [2,3], [2,4], [3,4], [3,5],[4,5],[4,6], [5,6], [1,5], [1,6], [2,6]]

# Parse options
properties = ("all", "under-closed", "semi-closed", "weakly-closed", "d-chordal", "closed", "almost-closed")
parser = OptionParser()
parser.add_option("-p", "--property", action="store", dest="property",
                  type="choice", choices=properties, default=properties[0],
                  help="Property to analyze. Valid properties are %s. Default property is %s" % (properties, properties[0]))
parser.add_option("-f") # ignore google colab option
(options, args) = parser.parse_args()

def exception (str, code=1):
  # TODO print prefix according to code, e.g. invalid input
  print(str, file=sys.stderr)
  sys.exit(code)

def isConsecutive (numList):
  return sorted(set(numList)) == list(range(min(numList), max(numList) + 1))

def loadInputMatrix ():
  matrix = []
  for line in sys.stdin:
    matrix.append([int(num) for num in line.split()])
  return matrix

# Load and validate matrix from stdin
# Create list of facets of the given simplicial complex; facet is d-dimensional face, 
# where d is the dimension of the simplicial complex

if os.isatty(sys.stdin.fileno()):
  list_of_facets = loadInputMatrix()

list_of_facets_flat = [item for sublist in list_of_facets for item in sublist]
if not isConsecutive(list_of_facets_flat):
  exception("Input matrix is not consecutive")

# load and validate verticies
number_of_vertices = max(list_of_facets_flat)
if number_of_vertices < 2 or number_of_vertices > 10:
  exception("Number of verticies must be in range <2,10>")

print('Number of vertices in our complex is "%s".' % (number_of_vertices))
print('Our complex contains the facets is "%s".' % (list_of_facets))
print('Choosen property is "%s".' % (options.property))


Number of vertices in our complex is "6".
Our complex contains the facets is "[[1, 2], [1, 3], [2, 3], [2, 4], [3, 4], [3, 5], [4, 5], [4, 6], [5, 6], [1, 5], [1, 6], [2, 6]]".
Choosen property is "all".


# Relabeling vertices (facets) function

In [ ]:
# Function that relabels vertices and therefore facets according to a given permutation 

def facets_relabeling(facets, perm):
  ## Relabaled facets setting to empty array
  relabeled_facets = []
  ## Number of vertices in a facet (dimension of the complex + 1)
  d = len(facets[1])
  ## Going througt all the facets of the complex and relabeling
  for k in range(0,len(facets)):
    ## Setting the k-th relabaled facet to empy array ready to be filled  
    relabeled_facet = []
    ## Going throuht all the vertices of the k-th facet 
    for l in range(0,d):
      ## Vertex before labeling
      vertex = facets[k][l]
      ## New label of the vertex
      relabaled_vertex = perm[vertex-1]
      ## Adding the new label 
      relabeled_facet.append(relabaled_vertex)
    ## Sorting the labels to get increasing labeling of the facet   
    relabeled_facet.sort()
    ## Adding the relabaled facet to the list of relabaled facets
    relabeled_facets.append(relabeled_facet)

  return relabeled_facets

Number of vertices in our complex is 6 .
Our complex contains the facets [[1, 2], [1, 3], [2, 3], [2, 4], [3, 4], [3, 5], [4, 5], [4, 6], [5, 6], [1, 5], [1, 6], [2, 6]] .


# Under-closed check
A pure $d$-dimensional simplicial complex $\Delta$ is under-closed if there is a vertex labeling of $\Delta$ such that for every $d$-face $F=a_0a_1\dots a_d$ (written with $a_0<a_1<\dots <a_d$) the complex $\Delta$ contains all faces of the form $a_0b_1b_2\dots b_d$ with $b_1\leq a_1$, $b_2\leq a_2$, $\dots$, $b_d\leq a_d$. 

In [ ]:
## Function creating list of all potential facets from the under-closed condition
## Recursively
def expand(input):
  output = []
  for i in range(input[0]+1, input[1]+1):
    if (len(input) == 2):
      output.append([input[0], i])
      continue
    input[1] = i
    expanded = expand(input[1:])
    for e in expanded:
      e.insert(0, input[0])
      output.append(e)
  return output

####################################################################

## Checking under-closed condition for a given labeling 
## Returns True if the labeling is under-closed
## Returns False if the labeling is NOT under-closed

def underclosed_labeling_check(facets_labeling):
  ## Going througt all the facets of the CS 
  for k in range(0,len(facets_labeling)):
  ## Generating list of potencial facets G for given F
    under_faces = expand(facets_labeling[k])
    for G in under_faces:
      ## If G not in the original list of facets --> not under-closed; return False
      if (G in facets_labeling)==False:
        #print('The labeling', facets_labeling, 'is NOT under-closed. For example the face', G, 'is missing.')
        return False
  ## If for all facets condition true --> under-closed; return True
  else:
    #print('The labeling', facets_labeling, 'is under-closed.')
    return True  

######################################################################

## Check UC for all labelings
## For every permutation on 'number_of_vertices' elements, we check the desired condition.
## Import to print all permutations  
from itertools import permutations 

def underclosed_SC(facets_list, number_of_vert):
  ## Get all permutations of [1, 2, ..., number_of_vertices] in the list named perms
  perms = list(permutations(list(range(1, number_of_vert+1))) )

  ## Go through all the permutations
  for perm in perms:
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS under-closed
    check = underclosed_labeling_check(new_labeling)
    if check == True:
      print('The labeling', new_labeling, 'makes the CS under-closed')
      return True
  else:
    print('The CS is NOT under-closed')
    return False
 
 ########################################################################

underclosed_SC(list_of_facets, number_of_vertices)

The CS is NOT under-closed


False

# Semi-closed check

A pure $d$-dimensional simplicial complex $\Delta$ is semi-closed if there is a vertex labeling of $\Delta$ such that for every $d$-face $F=a_0a_1\dots a_d$ (written with $a_0 < a_1<\dots < a_d$) at least one of the following conditions hold:
1. (*underclosed condition*) the complex $\Delta$ contains all faces of the form $a_0b_1b_2\dots b_d$ with $b_1\leq a_1$, $b_2\leq a_2$, $\dots$, $b_d\leq a_d$, or
2. the complex $\Delta$ contains all faces of the form $i_0i_1 \dots i_{d-1}a_d$ with $i_0\geq a_0$, $i_1\geq a_1$, $\dots$, $i_{d-1} \geq a_{d-1}$. 

In [ ]:
## Function creating list of all potential facets from the under-closed condition
## Recursively
def expand(input):
  output = []
  for i in range(input[0]+1, input[1]+1):
    if (len(input) == 2):
      output.append([input[0], i])
      continue
    input[1] = i
    expanded = expand(input[1:])
    for e in expanded:
      e.insert(0, input[0])
      output.append(e)
  return output

## Function creating list of all potential facets from the second condition
## Recursively

def expand_inv(input):
  inp_len=len(input)
  output = []
  for i in range(input[-1]-1, input[-2]-1, -1):
    if (len(input) == 2):
      output.append([i,input[1]])
      continue
    input[-2] = i
    expanded = expand_inv(input[:-1])
    for e in expanded:
      e.append(input[-1])
      output.append(e)
  return output

###########################################################################

## Checking semi-closed condition for a given labeling 
## Returns True if the labeling is semi-closed
## Returns False if the labeling is NOT semi-closed

def semi_closed_labeling_check(facets_labeling):
  ## Going througt all the facets of the CS 
  for k in range(0,len(facets_labeling)):
  ## Generating lists of potencial facets G for given F
    under_faces = expand(facets_labeling[k])
    above_faces = expand_inv(facets_labeling[k])
    ## Set under and semi_two to True.
    ## Will change to False if some face is missing  
    under=True
    semi_two=True
    for G in under_faces:
      ## If G not in the underclosed list of facets --> not underclosed; set under to False
      if (G in facets_labeling)==False:
        under=False
        G_under_false=G
        break
    for G in above_faces:
      ## If G not in the semi-closed list of facets --> not SM; set semi_two to False
      if (G in facets_labeling)==False:
        semi_two=False
        G_semi_false=G
        break  
    ## If both conditions for a given facet False, then the labeling NOT SM
    if under==False and semi_two==False:
      #print('The labeling', facets_labeling, 'is NOT semi-closed. One of the following facets would have to be present:', G_under_false, 'or', G_semi_false, 'for the facet', facets_labeling[k] )
      return False

  ## If at least one of the conditions for all the facets True, then the labeling is SM  
  else:
    #print('The labeling', facets_labeling, 'is semi-closed.')
    return True 
#######################################################################

## For every permutation on 'number_of_vertices' elements, we check the desired condition.
## Import to print all permutations  
from itertools import permutations 

def semi_closed_SC(facets_list, number_of_vert):
  ## Get all permutations of [1, 2, ..., number_of_vertices] in the list named perms
  perms = list(permutations(list(range(1, number_of_vert+1))) )

  ## Go through all the permutations
  for perm in perms:
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS underclosed
    check = semi_closed_labeling_check(new_labeling)
    if check == True:
      print('The labeling', new_labeling, 'makes the CS semi-closed')
      return True
  else:
    print('The CS is NOT semi-closed')
    return False
#########################################################################

semi_closed_SC(list_of_facets, number_of_vertices)

The labeling [[1, 2], [1, 3], [2, 3], [2, 6], [3, 6], [3, 5], [5, 6], [4, 6], [4, 5], [1, 5], [1, 4], [2, 4]] makes the CS semi-closed


True

# Weakly-closed check

A pure $d$-dimensional simplicial complex $\Delta$ is weakly-closed if there is a vertex labeling of $\Delta$ such that for every $d$-face $F=a_0a_1\dots a_d$ (written with $a_0<a_1<\dots <a_d$) and for every $g\notin F$ with $a_0<g<a_d$ there exists a $d$-face $G$ adjacent to $F$ containing $g$ such that either $\max G \neq \max F$ or $\min G \neq \min F$.

In [ ]:
## Function to check WC condition for a given facet 
def WC_for_a_facet(F, facets_labeling):
  ## Find all g such that a_0 < g < a_d
  b = [*range(F[0], F[-1]+1)]
  ## Filter those in F
  gs = list(set(b)-set(F)) 
  ## For each g with given condition
  for g in gs:
    ## Go through all the facets in the CS containig g and check max/min and adjecency
    g_ok=False
    for G in facets_labeling:
      if g in G and (F[0]!=G[0] or F[-1]!=G[-1]) and len(set(F).intersection(G))==len(F)-1:
        g_ok=True
        break
    if g_ok==False:
      #!!!print('Missing a suitable facet for ', g, 'adjecent to ', F)
      return False
  else:
      #print('WC condition ok for', F)
      return True 

#####################################################################
## Check if a given labeling has the weakly closed condition
def WC_labeling_check(facets_labeling): 
  ## Check all facet with the above function
  for F in facets_labeling:
    if WC_for_a_facet(F, facets_labeling)==False:
      #print('labeling', facets_labeling, 'is not WC')
      return False
  else:
    #print('labeling', facets_labeling, 'is WC')
    return True

###################################################################
## For every permutation on 'number_of_vertices' elements, we check the desired condition.
## Import to print all permutations  
from itertools import permutations 

def WC_SC(facets_list, number_of_vert):
  ## Get all permutations of [1, 2, ..., number_of_vertices] in the list named perms
  perms = list(permutations(list(range(1, number_of_vert+1))) )

  ## Go through all the permutations
  for perm in perms:
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS underclosed
    check = WC_labeling_check(new_labeling)
    if check == True:
      print('The labeling', new_labeling, 'makes the CS WC')
      return True
  else:
    print('The CS is NOT WC')
    return False

##################################################################

WC_SC(list_of_facets, number_of_vertices)

The CS is NOT WC


False

# d-chordal check
A pure $d$-dimensional simplicial complex $\Delta$ is d-chordal if there is a vertex labeling of $\Delta$ such that for every pair of $d$-faces $F=a_0a_1\dots a_d$ and $G=b_0b_1\dots b_d$ (written in increasing order) with  $a_d=b_d$, the complex $\Delta$ contains the whole $d$-skeleton of the simplex on $F \cup G$.

In [ ]:
## Function to check d-chordal condition for given pair of facets *with the same maximum*

## A tool to print all combinations of given length 
from itertools import combinations
 
## Note: not checking for the same max of F and G 
def d_chordal_for_pair_of_facets(F, G, facets_labeling):
 
  ## Do union of elements of F and G without repetitions
  union_F_G = list(set(F + G))
  ## Get all combinations of F union G of lenght len(F) == d-skeleton on F unoin G
  ## Already sorted!
  d_skeleton = [*combinations(union_F_G, len(F))]

  for H in d_skeleton:
    L=[*H]
    if L not in facets_labeling:
      # print('This labeling is not d-chordal. Missing face', L, 'for the pair', F, G)
      return False
  else:
    return True 

##############################################################################
## Fucntion to check if a given labeling is d-chordal
## Find all facet pairs with the same maximum and check d-chordality condition for them 
def d_chordal_for_labeling(facets_labeling):

  ## Going through all the facet pairs (F, G)
  for var in combinations(facets_labeling, 2): 
    ## If the maximum of F and G is the same, check the d-chord condition
    if var[0][-1] == var[1][-1]: 
      if d_chordal_for_pair_of_facets(var[0], var[1], facets_labeling) == False:
        #print('This labeling is not d-chordal. The trouble pair is', var[0], var[1] )
        return False 
      #print('ok pair')
  else:
     #print('The labeling', facets_labeling, 'is d-chordal')   
     return True

#############################################################################
## For every permutation on 'number_of_vertices' elements, we check the desired condition.
## Import to print all permutations  
from itertools import permutations 

def d_chordal_SC(facets_list, number_of_vert):
  ## Get all permutations of [1, 2, ..., number_of_vertices] in the list named perms
  perms = list(permutations(list(range(1, number_of_vert+1))) )

  ## Go through all the permutations
  for perm in perms:
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS underclosed
    check = d_chordal_for_labeling(new_labeling)
    if check == True:
      print('The labeling', new_labeling, 'makes the CS d-chordal')
      return True
  else:
    print('The CS is NOT d-chordal')
    return False

##############################################################################
d_chordal_SC(list_of_facets, number_of_vertices)

The CS is NOT d-chordal


False

# Closed check
 A pure $d$-dimensional simplicial complex $\Delta$ is closed if there exists a vertex labeling of $\Delta$ such that for every pair of $d$-faces $F=a_0a_1\dots a_d$ and $G=b_0b_1\dots b_d$ (written in increasing order) with  $a_i=b_i$ for some $i$, the complex $\Delta$ contains the full $d$-skeleton of the simplex on $F \cup G$.

In [ ]:
## Function to check closed condition for a given pair of facets

## A tool to print all combinations of given length 
from itertools import combinations 

## Note: not checking for the same a_i=b_i of F and G 
def closed_for_pair_of_facets(F, G, facets_labeling):
 
  ## Do union of elements of F and G without repetitions
  union_F_G = list(set(F + G))

  ## Get all combinations of F union G of lenght len(F) == d-skeleton on F unoin G
  ## Already sorted!
  d_skeleton = [*combinations(union_F_G, len(F))]
  
  for H in d_skeleton:
    L=[*H]
    if L not in facets_labeling:
     # print('This labeling is not closed. Missing face', L, 'for the pair', F, G)
      return False
  else:
    return True 

##############################################################################
## Fucntion to check if a given labeling is closed
## Find all facet pairs with the same i-th coordinate and check closed condition for them
def closed_for_labeling(facets_labeling):

  ## Going through all the facet pairs (F, G)
  n=len(facets_labeling[0])
  for var in combinations(facets_labeling, 2): 
    ## If the i-th coordinate of F and G is the same, check the closed condition
    for i in range(0,n):
      if var[0][i] == var[1][i]: 
        if closed_for_pair_of_facets(var[0], var[1], facets_labeling) == False:
          #print('This labeling', facets_labeling,' is not closed. The trouble pair is', var[0], var[1] )
          return False 
        break
  else:
     #print('The labeling', facets_labeling, 'is closed')   
     return True

#############################################################################
## For every permutation on 'number_of_vertices' elements, we check the desired condition.
## Import to print all permutations  
from itertools import permutations 

def closed_SC(facets_list, number_of_vert):
  ## Get all permutations of [1, 2, ..., number_of_vertices] in the list named perms
  perms = list(permutations(list(range(1, number_of_vert+1))) )

  ## Go through all the permutations
  for perm in perms:
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS underclosed
    check = closed_for_labeling(new_labeling)
    if check == True:
      print('The labeling', new_labeling, 'makes the CS closed')
      return True
  else:
    print('The CS is NOT closed')
    return False

##############################################################################
closed_SC(list_of_facets, number_of_vertices)

The CS is NOT closed


False

# Almost-closed CS check
Let $\Delta$ be a pure $d$-dimensional simplicial complex with $n$ vertices. The complex $\Delta$ is called almost-closed if there exists a labeling $1, \ldots, n$ of its vertices such that  for any $d$-face $F=a_0 a_1 \cdots a_d$ of $\Delta$, the complex $\Delta$ contains the whole $d$-skeleton of the simplex with vertex set $\{a_0, a_0 +1, a_0 + 2, \ldots, a_d\}$.

In [ ]:
## Function to check almost-closed condition for a given facet

def almost_closed_for_facet(F, facets_labeling):
 
  ## Do union of elements of F and G without repetitions
  set_for_F = list(range(F[0], F[-1]+1))
  # print('facet', F, 'set', set_for_F)
  ## A tool to print all combinations of given length 
  from itertools import combinations 
    
  ## Get all combinations of {a_0, a_0+1, ..., a_d} of lenght len(F) = d-skeleton on the set of vertices
  ## Already sorted!
  d_skeleton = [*combinations(set_for_F, len(F))]
  
  #print(d_skeleton)

  for H in d_skeleton:
    L=[*H]
    if L not in facets_labeling:
      #print('This labeling is not almost-closed. Missing face', L, 'for the facet', F)
      return False
  else:
    #print('The facet', F ,'passed the almost-closed test')
    return True 

##############################################################################
## Fucntion to check if a given labeling is almost-closed
def almost_closed_for_labeling(facets_labeling):
  #print(facets_labeling)
  ## Going through all the facets F
  n=len(facets_labeling[0])
  #print('dimension is', n-1)
  for var in facets_labeling: 
    #print(var)
    ## check the almost-closed condition
    if almost_closed_for_facet(var, facets_labeling) == False:
          #print('This labeling', facets_labeling,' is not almost-closed. The trouble facet is', var)
          return False 
  else:
     #print('The labeling', facets_labeling, 'is almost-closed')   
     return True

##############################################################################
## For every permutation on 'number_of_vertices' elements, we check the desired condition.
## Import to print all permutations  
from itertools import permutations 

def almost_closed_SC(facets_list, number_of_vert):
  ## Get all permutations of [1, 2, ..., number_of_vertices] in the list named perms
  perms = list(permutations(list(range(1, number_of_vert+1))) )

  ## Go through all the permutations
  for perm in perms:
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS underclosed
    check = almost_closed_for_labeling(new_labeling)
    if check == True:
      print('The labeling', new_labeling, 'makes the CS almost-closed')
      return True
  else:
    print('The CS is NOT almost-closed')
    return False

##############################################################################
almost_closed_SC(list_of_facets, number_of_vertices)

The labeling [[1, 2, 3, 4], [1, 2, 3, 5], [1, 2, 4, 5], [1, 3, 4, 5], [2, 3, 4, 5], [2, 3, 4, 6], [2, 3, 5, 6], [2, 4, 5, 6], [3, 4, 5, 6]] makes the CS almost-closed


True